Get data from github

In [1]:
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/train.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/valid.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/test.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_classifier.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/requirements.txt

--2023-06-25 21:36:05--  https://raw.githubusercontent.com/shitkov/bert4classification/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9340188 (8.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   8.91M  1.81MB/s    in 5.9s    

2023-06-25 21:36:12 (1.51 MB/s) - ‘train.csv’ saved [9340188/9340188]

--2023-06-25 21:36:12--  https://raw.githubusercontent.com/shitkov/bert4classification/main/valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 518234 (506K) [text/plain]
Savin

Installing Packages

In [2]:
!pip install sentencepiece==0.1.95

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentencepiece: filename=sentencepiece-0.1.95-cp310-cp310-linux_x86_64.whl size=1546214 sha256=7aa72d99c208bab3d110825c0b157756d2ff26dc9c717de7a4dc3949a8fe9d0e
  Stored in directory: /root/.cache/pip/wheels/ef/a4/01/5a500fc0c5a38917ef408c245eb40b7ac96f4a30fc6a346a4c
Successfully built sentencepiece


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 5.7 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 10.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.8 MB/s eta 0:00:00
  Created wheel for pyyaml: filename=PyYAML-6.0-cp38-cp38-macosx_13_0_arm64.whl size=45336 sha256=389e282d9f6262f02845f59696ac676b034f534cff3dec39e6054d05b8f80351
  Stored in directory: /Users/buchkovv/Library/Caches/pip/wheels/52/84/66/50912fd7bf1639a31758e40bd4312602e104a8eca1

In [1]:
import pandas as pd

In [2]:
!pip install pymorphy2

In [3]:
from bert_dataset import CustomDataset
from bert_regressor import BertRegressor

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
RANDOM_STATE = 12

In [5]:
BACKTEST_CCY = 'USDRUB'
BACKTEST_DAYS = 5
YEARS = [2021, 2022]

# PATH_TEXTS = 'drive/MyDrive/data/telegram'
# PATH_OPT_PNL = 'drive/MyDrive/data/pnl/spot'

PATH_TEXTS = 'data/telegram'
PATH_OPT_PNL = 'data/pnl/spot'

N_EPOCHS = 10

In [6]:
import os

# Get all available data_sources
sources = os.listdir(PATH_TEXTS)
sources

['cbonds.csv',
 'themovchans.csv',
 'headlines_QUANTS.csv',
 'War_Wealth_Wisdom.csv',
 'mmi.csv',
 'vts.csv',
 'signal.csv',
 'rshb_invest.csv',
 'Alfa_Wealth.csv',
 'sky_bond.csv',
 'bitkogan.csv']

In [7]:
# Create target variable dataframe
pnl = pd.DataFrame()
for year in YEARS:
    pnl_year = pd.read_csv(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    pnl = pd.concat([pnl, pnl_year], axis=0)

pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl.dropna(subset=['pnl'], inplace=True)
pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
pnl

,Bid,Ask,date_start,Mid,pnl,pnl_sign
5,120.600105,120.751750,2022-03-10,120.675928,0.257866,1
6,116.991277,119.658190,2022-03-11,118.324734,0.234001,1
7,114.526205,116.354587,2022-03-14,115.440396,0.004175,1
8,111.299008,112.598900,2022-03-15,111.948954,0.050668,1
9,108.821595,109.377133,2022-03-16,109.099364,-0.067747,0
...,...,...,...,...,...,...
153,61.484698,61.978255,2022-10-18,61.731476,-0.016085,0
154,61.588507,62.014587,2022-10-19,61.801547,-0.038277,0
155,61.600813,62.025918,2022-10-20,61.813365,-0.037216,0
156,61.144888,61.667988,2022-10-21,61.406438,-0.034316,0


In [8]:
import datetime as dt
from typing import List, Tuple, Union


def _binary_search_by_date(array: List[Tuple[dt.datetime, float]], date_x: dt.datetime) -> Union[int, None]:
    """
    Searches for the index of date_x in the array via binary search.

        Parameters:
            array (list) : A sorted array of (date, float_value) tuples
            date_x (datetime.datetime) : Date to search for

        Returns:
            index_x (int): Index of the searched date in the array.
    """
    left = 0
    right = len(array) - 1

    while left <= right:
        mid = left + (right - left) // 2

        if array[mid - 1][0] <= date_x <= array[mid][0]:
            return mid
        elif date_x > array[mid - 1][0] and date_x > array[mid][0]:
            left = mid + 1
        else:
            right = mid - 1

    return None


def binary_search_time_series(time_series: List[Tuple[dt.datetime, float]], date_start: dt.datetime,
                              date_end: dt.datetime) -> Union[List[Tuple[dt.datetime, float]], None]:
    """
    Searches for the part of the time series that is contained inside [date_start; date_end] period via binary search.

        Parameters:
            time_series (list) : A sorted array of (date, float_value) tuples
            date_start (datetime.datetime) : Starting date of the searched period
            date_end (datetime.datetime) : Ending date of the searched period

        Returns:
            time_series_data (list): Part of the time series that is contained inside [date_start; date_end] period.
    """

    if date_start <= date_end:
        left_index = _binary_search_by_date(time_series, date_start)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_end)

        if right_index is None:
            return None

        right_index += left_index
    else:
        left_index = _binary_search_by_date(time_series, date_end)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_start)

        if right_index is None:
            return None

        right_index += left_index

    return time_series[left_index:right_index]

In [9]:
pnl_sign_ts = [(pd.to_datetime(row['date_start']).to_pydatetime(), row['pnl_sign']) for  _, row in pnl.iterrows()]
pnl_sign_ts

[(datetime.datetime(2022, 3, 10, 0, 0), 1),
 (datetime.datetime(2022, 3, 11, 0, 0), 1),
 (datetime.datetime(2022, 3, 14, 0, 0), 1),
 (datetime.datetime(2022, 3, 15, 0, 0), 1),
 (datetime.datetime(2022, 3, 16, 0, 0), 0),
 (datetime.datetime(2022, 3, 17, 0, 0), 0),
 (datetime.datetime(2022, 3, 18, 0, 0), 0),
 (datetime.datetime(2022, 3, 21, 0, 0), 0),
 (datetime.datetime(2022, 3, 22, 0, 0), 0),
 (datetime.datetime(2022, 3, 23, 0, 0), 0),
 (datetime.datetime(2022, 3, 24, 0, 0), 0),
 (datetime.datetime(2022, 3, 25, 0, 0), 0),
 (datetime.datetime(2022, 3, 28, 0, 0), 0),
 (datetime.datetime(2022, 3, 29, 0, 0), 0),
 (datetime.datetime(2022, 3, 30, 0, 0), 0),
 (datetime.datetime(2022, 3, 31, 0, 0), 0),
 (datetime.datetime(2022, 4, 1, 0, 0), 0),
 (datetime.datetime(2022, 4, 4, 0, 0), 0),
 (datetime.datetime(2022, 4, 5, 0, 0), 0),
 (datetime.datetime(2022, 4, 6, 0, 0), 0),
 (datetime.datetime(2022, 4, 7, 0, 0), 0),
 (datetime.datetime(2022, 4, 8, 0, 0), 0),
 (datetime.datetime(2022, 4, 11, 0, 0)

In [10]:
initial_date = pnl_sign_ts[0][0]

def get_pnl_sign(row):
    date_x = pd.to_datetime(row['date']).to_pydatetime()
    pnl_sign_key = _binary_search_by_date(pnl_sign_ts, date_x)

    if pnl_sign_key is not None and date_x >= initial_date:
        return pnl_sign_ts[_binary_search_by_date(pnl_sign_ts, date_x)][1]

In [11]:
# Generate dataframes
df = pd.DataFrame()

for s in sources:
    if s != '.gitignore':
        source_data = pd.read_csv(f'{PATH_TEXTS}/{s}')
        df = pd.concat([df, source_data], axis=0)

df.head()

,id,date,text
0,9,2017-07-28T10:56:14,Cbonds.ru запустило канал в Telegram. Планиру...
1,10,2017-08-03T15:28:17,Облигационный бюллетень Cbonds – все данные о ...
2,11,2017-08-03T15:41:02,Cbonds prepared monthly report: CBONDS GLOBAL ...
3,12,2017-08-03T18:11:32,Дайджест Cbonds от 3 августа:«Группа Компаний...
4,13,2017-08-04T12:42:41,Совкомбанк открыл книгу заявок на вторичное ра...


In [12]:
df['pnl_sign'] = df.apply(lambda row: get_pnl_sign(row), axis=1)
df.dropna(subset=['pnl_sign'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,id,date,text,pnl_sign
0,10884,2022-03-10T09:43:31,УТРЕННИЙ ДАЙДЖЕСТ ⭕️ О режиме работы Московск...,1.0
1,10885,2022-03-10T10:38:48,💻 JPMorgan Chase & Co. отказался от роли конс...,1.0
2,10886,2022-03-10T11:38:45,#Еврооблигации #ОФЗ 📝Выплаты купонов по ОФЗ...,1.0
3,10887,2022-03-10T11:57:48,🔨🔁« Petropavlovsk» уведомил о возможной допэми...,1.0
4,10888,2022-03-10T13:09:36,​​ 📝На Cbonds доступны параметры облигаций и ц...,1.0


In [13]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/buchkovv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm
from tqdm.gui import tqdm as tqdm_gui

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)

    return ' '.join(remove_stopwords(lemmas))

In [15]:
lemmas = list(tqdm(map(clean_text, df['text']), total=len(df['text'].to_list())))

  0%|          | 0/15626 [00:00<?, ?it/s]

In [16]:
df['lemmas'] = lemmas
df = df[df['lemmas'] != '']
df.head()

,id,date,text,pnl_sign,lemmas
0,10884,2022-03-10T09:43:31,УТРЕННИЙ ДАЙДЖЕСТ ⭕️ О режиме работы Московск...,1.0,утренний дайджест режим работа московский бирж...
1,10885,2022-03-10T10:38:48,💻 JPMorgan Chase & Co. отказался от роли конс...,1.0,jpmorgan chase отказаться роль консультант yan...
2,10886,2022-03-10T11:38:45,#Еврооблигации #ОФЗ 📝Выплаты купонов по ОФЗ...,1.0,еврооблигация выплата купон евробондать россия...
3,10887,2022-03-10T11:57:48,🔨🔁« Petropavlovsk» уведомил о возможной допэми...,1.0,petropavlovsk уведомить возможный допэмиссия а...
4,10888,2022-03-10T13:09:36,​​ 📝На Cbonds доступны параметры облигаций и ц...,1.0,cbonds доступный параметр облигация цена прода...


In [17]:
df.shape

(15618, 5)

In [18]:
df['pnl_sign'].sum() / df.shape[0]

0.3740555768984505

Initialize BERT classifier

In [20]:
classifier = BertRegressor(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        epochs=N_EPOCHS,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Prepare data and helpers for train and evlauation

In [26]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.8 MB/s eta 0:00:00


In [21]:
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df[['lemmas', 'pnl']], target = 'pnl',
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)

KeyError: "['pnl'] not in index"

In [28]:
classifier.preparation(
        X_train=X_train['lemmas'].tolist(),
        y_train=y_train.tolist(),
        X_valid=X_valid['lemmas'].tolist(),
        y_valid=y_valid.tolist()
    )

Train loop

In [ ]:
classifier.train()

Epoch 1/10
Train loss 0.6978413709351825 accuracy 0.5258342748491278
Val loss 0.6874937335755249 accuracy 0.5407848324514991
----------
Epoch 2/10


Check test data

In [ ]:
predictions = [classifier.predict(t) for t in X_test['lemmas'].tolist()]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(y_test.tolist(), predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test.tolist(), predictions, labels=['pos pnl', 'neg pnl'])
cm